<a href="https://colab.research.google.com/github/qasimzee/isic2024/blob/main/isic_2024_challenge_table_dnn_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install scikit-learn==1.2.2
!pip install numpy==1.26.4

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!ls "/content/drive/MyDrive/kaggle/isic-2024-data"

cnn.pkl			 sample_submission.csv	test-image.hdf5		  train-image.hdf5
logistic_regression.pkl  scaler.pkl		test-metadata.csv	  train-metadata.csv
random_forest_model.pkl  sequential_dnn.pkl	test-metadata.csv.gsheet  train-metadata-v1.csv


In [11]:
METADATA_FILE = "/content/drive/MyDrive/kaggle/isic-2024-data/train-metadata.csv"
metadata_df = pd.read_csv(METADATA_FILE)


<ipython-input-11-6d834c4c6865>:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_df = pd.read_csv(METADATA_FILE)


In [12]:
TESTDATA_FILE = "/content/drive/MyDrive/kaggle/isic-2024-data/test-metadata.csv"
test_data_df = pd.read_csv(TESTDATA_FILE)
test_data_df

,isic_id,patient_id,target,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,attribution,copyright_license
0,ISIC_0015657,IP_6074337,1,45,male,posterior torso,2.70,TBP tile: close-up,3D: XP,22.80433,...,0.304827,1.281532,2.299935,0.479339,20,-155.06510,1511.222000,113.980100,Memorial Sloan Kettering Cancer Center,CC-BY
1,ISIC_0015729,IP_1664139,0,35,female,lower extremity,2.52,TBP tile: close-up,3D: XP,16.64867,...,0.000000,1.271940,2.011223,0.426230,25,-112.36924,629.535889,-15.019287,"Frazer Institute, The University of Queensland...",CC-BY
2,ISIC_0015740,IP_7142616,1,65,male,posterior torso,3.16,TBP tile: close-up,3D: XP,24.25384,...,0.230742,1.080308,2.705857,0.366071,110,-84.29282,1303.978000,-28.576050,FNQH Cairns,CC-BY


In [13]:
train_df_unique_cols = set(metadata_df.columns) - set(test_data_df.columns)
print("Columns in train_df but not in test_df:", train_df_unique_cols)

# Columns in test_df but not in train_df
test_df_unique_cols = set(test_data_df.columns) - set(metadata_df.columns)
print("Columns in test_df but not in train_df:", test_df_unique_cols)

Columns in train_df but not in test_df: {'mel_thick_mm', 'iddx_5', 'iddx_4', 'mel_mitotic_index', 'lesion_id', 'iddx_3', 'iddx_full', 'iddx_1', 'tbp_lv_dnn_lesion_confidence', 'iddx_2'}
Columns in test_df but not in train_df: set()


In [14]:
train_df, test_df = train_test_split(metadata_df, test_size=0.2, random_state=42)

Y_test = test_df['target']


In [15]:
Y = train_df["target"]

dropped_columns = ["patient_id", "lesion_id", "iddx_full", "iddx_1", "iddx_2", "iddx_3", "iddx_4", "iddx_5", "mel_mitotic_index", "mel_thick_mm", "tbp_lv_dnn_lesion_confidence"]
X = train_df.drop(columns=dropped_columns)

# Replace NaN values
X['sex'] = X['sex'].replace(pd.NA, 'unknown')
X['anatom_site_general'] = X['anatom_site_general'].replace(pd.NA, 'unknown')

X.fillna(0, inplace=True)

In [16]:
# Identify columns with mixed types

for column in X.columns:
  if X[column].apply(type).nunique() > 1:
    print(f"{column}: {X[column].apply(type).unique()}")


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Identify categorical features
categorical_columns = X.select_dtypes(include=['object']).columns
numerical_columns = X.select_dtypes(include=['number']).columns

# Create a ColumnTransformer with OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_columns),  # One-hot encode categorical columns
        ('num', StandardScaler(), numerical_columns)  # Scale numerical columns
    ],
    remainder='passthrough'
)

X_train_encoded = preprocessor.fit_transform(X)


# Assuming you've already preprocessed your categorical variables
model = Sequential([
    Dense(64, input_shape=(X_train_encoded.shape[1],), activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
class_weight = {0: 1, 1: 100}  # Adjust the weight ratio to balance the classes

model.fit(X_train_encoded, Y, epochs=2, batch_size=32)



In [ ]:
# test_df['sex'] = test_df['sex'].replace(pd.NA, 'unknown')
# test_df['anatom_site_general'] = test_df['anatom_site_general'].replace(pd.NA, 'unknown')
# test_df['lesion_id'] = test_df['lesion_id'].replace(pd.NA, 'unknown')
# test_df['iddx_2'] = test_df['iddx_2'].replace(pd.NA, 'unknown')
# test_df['iddx_3'] = test_df['iddx_3'].replace(pd.NA, 'unknown')
# test_df['iddx_4'] = test_df['iddx_4'].replace(pd.NA, 'unknown')
# test_df['mel_mitotic_index'] = test_df['mel_mitotic_index'].replace(pd.NA, 'unknown')

# test_df.fillna(0, inplace=True)
# # Identify columns with mixed types
# for column in test_df.columns:
#   if test_df[column].apply(type).nunique() > 1:
#     print(f"{column}: {test_df[column].unique()}")


test_isic_ids = test_df['isic_id']
test_df = test_df.drop(columns=dropped_columns)
test_df.head()

In [ ]:
# Make predictions on the test data
X_test_encoded = preprocessor.fit_transform(test_df)
y_pred_proba = model.predict(X_test_encoded)  # Probability of positive class (e.g., malignant)

# Create a DataFrame to map predictions back to 'isic_id'
predictions_df = pd.DataFrame({
    'isic_id': test_isic_ids,
    'target': y_pred_proba.flatten()
})


predictions_df


In [ ]:
y_pred_proba = np.nan_to_num(y_pred_proba, nan=0)


In [ ]:
y_pred_proba.max()

In [ ]:
Y_test

In [ ]:
import numpy as np
from sklearn.metrics import roc_curve, auc


# Compute the ROC curve
fpr, tpr, thresholds = roc_curve(Y_test, y_pred_proba)

# Specify the range of TPR you are interested in (e.g., above 80%)
# Filter the points where TPR is above 0.8
tpr_threshold = 0.8
valid_range = tpr >= tpr_threshold

# Compute pAUC within the valid range
partial_auc = auc(fpr[valid_range], tpr[valid_range])

print(f"Partial AUC for TPR > 80%: {partial_auc:.2f}")


In [ ]:
import pickle
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

# Assume you have trained your pipeline as described before

# Save the entire pipeline to a pickle file
with open('/content/drive/MyDrive/kaggle/isic-2024-data/sequential_dnn.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Model pipeline saved successfully!")
